In [2]:
import sys
import os

# プロジェクトルートをsys.pathに追加
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), "../../.."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PYTHONPATH set to:", project_root)

PYTHONPATH set to: c:\Users\ndhaj\Desktop\TCI


In [17]:
# =============================
# [0] 共通Importと初期化
# =============================
from PtychoEP.backend.backend import set_backend, np
set_backend("numpy")

from PtychoEP.ptycho.core import Ptycho
from PtychoEP.utils.io_utils import load_data_image
from PtychoEP.ptycho.aperture_utils import circular_aperture
from PtychoEP.ptycho.scan_utils import generate_spiral_scan_positions
from PtychoEP.ptycho.noise import GaussianNoise
from PtychoEP.ptycho.visualize import compute_illumination
from PtychoEP.rng.rng_utils import get_rng
from PtychoEP.classic_engines.pie import PIE
from PtychoEP.ptychoep.core import PtychoEP

import matplotlib.pyplot as plt


# =============================
# [1] Seed指定
# =============================
data_seed = 0
init_seed = 0
rng_data = get_rng(data_seed)
rng_init = get_rng(init_seed)

# =============================
# [2] 複素オブジェクトとプローブ生成
# =============================
amp = load_data_image("cameraman.png")
phs = load_data_image("eagle.png")
obj_true = amp * np().exp(1j * np().pi / 2 * phs)

probe = circular_aperture(size=64, r=0.47)

# =============================
# [3] Ptychoオブジェクトとデータ生成
# =============================
ptycho = Ptycho()
ptycho.set_object(obj_true)
ptycho.set_probe(probe)

positions = generate_spiral_scan_positions(
    image_size=512,
    probe_size=64,
    step=16.93,
    num_points=1000
)
ptycho.forward_and_set_diffraction(positions)
GaussianNoise(var=11.8e-5, seed=data_seed) @ ptycho

# 観測率を計算
_, alpha = compute_illumination(ptycho)
print(f"[INFO] Sampling ratio (alpha): {alpha:.2f}\n Scan Number (J) : {len(ptycho._diff_data)}\n Noise Stats : {ptycho.noise_stats}")

[INFO] Sampling ratio (alpha): 4.00
 Scan Number (J) : 225
 Noise Stats : {'type': 'Gaussian', 'var': 0.000118, 'snr_mean_db': 29.98586654663086}
